In [7]:
!pip install pandas

You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [8]:
!pip install psycopg2 #install postgres driver

Invalid requirement: '#install'
Traceback (most recent call last):
  File "C:\Users\daan\Anaconda3\lib\site-packages\pip\_vendor\packaging\requirements.py", line 90, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "C:\Users\daan\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1632, in parseString
    raise exc
  File "C:\Users\daan\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1622, in parseString
    loc, tokens = self._parse( instring, 0 )
  File "C:\Users\daan\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File "C:\Users\daan\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 3395, in parseImpl
    loc, exprtokens = e._parse( instring, loc, doActions )
  File "C:\Users\daan\Anaconda3\lib\site-packages\pip\_vendor\pyparsing.py", line 1379, in _parseNoCache
    loc,tokens = self.parseImpl( instring, preloc, doActions )
  File 

In [9]:
DB_HOST = '89.145.163.87'
DB_PORT = 5432
DB_USERNAME = 'group11'
DB_PASSWORD = 'tsw2020'
DB_NAME = 'tweets'

In [10]:
regions = {
    'italy': ['italy1', 'italy2'],
    'germany': ['germany1', 'germany2', 'germany3']
}

In [11]:
def region_sql_query(region):
    return """
SELECT
tweet_id,
inserted,

location_query,
location_id,

tweet_body->'created_at' as created_at,
tweet_body->'user'->'id_str' as user_id_str,
tweet_body->'user'->'screen_name' as user_screen_name,
tweet_body->'user'->'followers_count' as followers_count,

tweet_body->'entities'->'user_mentions' as user_mentions,

tweet_body->'in_reply_to_status_id' as in_reply_to_status_id,
tweet_body->'in_reply_to_status_id_str' as in_reply_to_status_id_str,
tweet_body->'in_reply_to_user_id_str' as in_reply_to_user_id_str,
tweet_body->'in_reply_to_screen_name' as in_reply_to_screen_name,

tweet_body->'retweeted_status'->'user'->>'id_str' as retweeted_user_id_str,
tweet_body->'retweeted_status'->'user'->>'screen_name' as retweeted_user_screen_name,

tweet_body->'metadata'->'result_type' as result_type,
tweet_body->>'text' as text
FROM tweet
WHERE location_id = '{}'
;
""".format(region)

In [12]:
import psycopg2
import pandas as pd

def tweets_to_pickle(region):
    # connect to DB
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USERNAME, password=DB_PASSWORD)
    cur = conn.cursor()
    
    circles = regions[region]
    print(circles)
    
    df_container = list()
    for r in circles:
        sql_query = region_sql_query(r)
        circle_tweets = pd.read_sql_query(sql_query, conn)
        circle_tweets.set_index('tweet_id')
        df_container.append(circle_tweets)

    tweets_database = pd.concat(df_container)
    print("Dataframe shape: {}".format(tweets_database.shape))
    print(tweets_database.info(memory_usage=True))
    
    cur.close()
    conn.close()
    
    tweets_database.to_pickle(path='tweets_database_{}.pickle'.format(region))

tweets_to_pickle('italy')
tweets_to_pickle('germany')

['italy1', 'italy2']
Dataframe shape: (139154, 17)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 139154 entries, 0 to 16800
Data columns (total 17 columns):
tweet_id                      139154 non-null object
inserted                      139154 non-null datetime64[ns]
location_query                139154 non-null object
location_id                   139154 non-null object
created_at                    139154 non-null object
user_id_str                   139154 non-null object
user_screen_name              139154 non-null object
followers_count               139154 non-null int64
user_mentions                 139154 non-null object
in_reply_to_status_id         2824 non-null float64
in_reply_to_status_id_str     2824 non-null object
in_reply_to_user_id_str       3255 non-null object
in_reply_to_screen_name       3255 non-null object
retweeted_user_id_str         109540 non-null object
retweeted_user_screen_name    109540 non-null object
result_type                   139154 non-nul